# Credit Risk Resampling Techniques

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import numpy as np
import pandas as pd
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [6]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [7]:
# Load the data
file_path = 'resources/LoanStats_2019Q1.csv'
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [43]:
# print(df.dtypes.to_string())
# print()
# print(df["loan_amnt"].dtypes)

# Find non-numeric columns
for column in df:
    if df[column].dtypes != "float64":
        print('Column Name : ', column, " -- is type -- ", df[column].dtypes)
        print(df[column][0])

Column Name :  home_ownership  -- is type --  object
RENT
Column Name :  verification_status  -- is type --  object
Source Verified
Column Name :  issue_d  -- is type --  object
Mar-2019
Column Name :  loan_status  -- is type --  object
low_risk
Column Name :  pymnt_plan  -- is type --  object
n
Column Name :  initial_list_status  -- is type --  object
w
Column Name :  next_pymnt_d  -- is type --  object
May-2019
Column Name :  application_type  -- is type --  object
Individual
Column Name :  hardship_flag  -- is type --  object
N
Column Name :  debt_settlement_flag  -- is type --  object
N


In [60]:
# Inspect columns with month/date information
print(df["issue_d"].unique())
print()
print(df["next_pymnt_d"].unique())

['Mar-2019' 'Feb-2019' 'Jan-2019']

['May-2019' 'Apr-2019']


### Convert months to chronologically ordered numbers first

In [61]:
# Create dictionary to manually rename months 
months_num = {
    "Jan-2019": 1,
    "Feb-2019": 2,
    "Mar-2019": 3,
    "Apr-2019": 4,
    "May-2019": 5
}

In [62]:
# copy df before altering
dates_df = df.copy()

In [64]:
# Encode issue_d using dictionary and check
dates_df["issue_d_num"] = dates_df["issue_d"].apply(lambda x: months_num[x])
dates_df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,issue_d_num
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N,3
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N,3
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N,3
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N,3
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N,3


In [65]:
# Encode next_pymnt_d using dictionary and check
dates_df["pymnt_d_num"] = dates_df["next_pymnt_d"].apply(lambda x: months_num[x])
dates_df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,issue_d_num,pymnt_d_num
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N,3,5
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N,3,5
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N,3,5
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N,3,5
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N,3,5


In [66]:
# Drop original month columns from dataframe
dates_df = dates_df.drop(["issue_d", "next_pymnt_d"], axis=1)
dates_df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,dti,delinq_2yrs,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,issue_d_num,pymnt_d_num
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,low_risk,n,27.24,0.0,...,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N,3,5
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,low_risk,n,20.23,0.0,...,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N,3,5
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,low_risk,n,24.26,0.0,...,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N,3,5
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,low_risk,n,31.44,0.0,...,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N,3,5
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,low_risk,n,18.76,0.0,...,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N,3,5


In [69]:
# Check again for non-numeric columns
for column in dates_df:
    if (dates_df[column].dtypes != "float64") and (dates_df[column].dtypes != "int64"):
        print('Column Name : ', column, " -- is type -- ", dates_df[column].dtypes)
        print(dates_df[column][0])

Column Name :  home_ownership  -- is type --  object
RENT
Column Name :  verification_status  -- is type --  object
Source Verified
Column Name :  loan_status  -- is type --  object
low_risk
Column Name :  pymnt_plan  -- is type --  object
n
Column Name :  initial_list_status  -- is type --  object
w
Column Name :  application_type  -- is type --  object
Individual
Column Name :  hardship_flag  -- is type --  object
N
Column Name :  debt_settlement_flag  -- is type --  object
N


### Use loop to convert remaining object columns to numeric

In [70]:
# Convert string values to numeric values
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
loans_df = dates_df.copy()

for column in loans_df:
    if loans_df[column].dtypes == "object":
        loans_df[column] = le.fit_transform(loans_df[column]) 

loans_df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,dti,delinq_2yrs,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,issue_d_num,pymnt_d_num
0,10500.0,0.1719,375.35,3,66000.0,1,1,0,27.24,0.0,...,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0,3,5
1,25000.0,0.2000,929.09,1,105000.0,2,1,0,20.23,0.0,...,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0,3,5
2,20000.0,0.2000,529.88,1,56000.0,2,1,0,24.26,0.0,...,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0,3,5
3,10000.0,0.1640,353.55,3,92000.0,2,1,0,31.44,0.0,...,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0,3,5
4,22000.0,0.1474,520.39,1,52000.0,0,1,0,18.76,0.0,...,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0,3,5


In [74]:
# Check for non-numeric columns one final time
print(loans_df.dtypes.to_string())
print()
print("Double check with loop (Blank is good): ")
print()
for column in loans_df:
    if (loans_df[column].dtypes != "float64") and (loans_df[column].dtypes != "int64"):
        print('Column Name : ', column, " -- is type -- ", loans_df[column].dtypes)
        print(loans_df[column][0])

loan_amnt                     float64
int_rate                      float64
installment                   float64
home_ownership                  int64
annual_inc                    float64
verification_status             int64
loan_status                     int64
pymnt_plan                      int64
dti                           float64
delinq_2yrs                   float64
inq_last_6mths                float64
open_acc                      float64
pub_rec                       float64
revol_bal                     float64
total_acc                     float64
initial_list_status             int64
out_prncp                     float64
out_prncp_inv                 float64
total_pymnt                   float64
total_pymnt_inv               float64
total_rec_prncp               float64
total_rec_int                 float64
total_rec_late_fee            float64
recoveries                    float64
collection_recovery_fee       float64
last_pymnt_amnt               float64
collections_

# Split the Data into Training and Testing

In [85]:
# Create our features
X = loans_df.drop("loan_status", axis=1)


# Create our target
y = loans_df["loan_status"]

In [86]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,issue_d_num,pymnt_d_num
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,0.0,21.778153,0.217766,0.497697,...,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0,1.726172,4.616839
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.0,20.199244,0.718367,0.758122,...,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0,0.743862,0.486161
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0,1.000000,4.000000
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,0.0,13.890000,0.000000,0.000000,...,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0,1.000000,4.000000
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,0.0,19.760000,0.000000,0.000000,...,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0,2.000000,5.000000
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,0.0,26.660000,0.000000,1.000000,...,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0,2.000000,5.000000
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,0.0,999.000000,18.000000,5.000000,...,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0,3.000000,5.000000


In [87]:
# Check the balance of our target values
y.value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [88]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [89]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({1: 51352, 0: 51352})

In [90]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [94]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = model.predict(X_test)

balanced_accuracy_score(y_test, y_pred)

0.6605337797277316

In [95]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[   62,    25],
       [ 6703, 10415]])

In [116]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print("Random Oversampler")
print("--------------------")
print("Imbalanced Classification Report:")
print(classification_report_imbalanced(y_test, y_pred))
print("Balanced Accuracy Score:")
balanced_accuracy_score(y_test, y_pred)

Random Oversampler
--------------------
Imbalanced Classification Report:
                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.71      0.61      0.02      0.66      0.44        87
          1       1.00      0.61      0.71      0.76      0.66      0.43     17118

avg / total       0.99      0.61      0.71      0.75      0.66      0.43     17205

Balanced Accuracy Score:


0.6605337797277316

### SMOTE Oversampling

In [98]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled_s, y_resampled_s = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train, y_train
)
Counter(y_resampled_s)

Counter({1: 51352, 0: 51352})

In [99]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled_s, y_resampled_s)

LogisticRegression(random_state=1)

In [100]:
# Calculated the balanced accuracy score
y_pred_s = model.predict(X_test)

balanced_accuracy_score(y_test, y_pred_s)

0.6172137818227235

In [102]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_s)

array([[   50,    37],
       [ 5825, 11293]])

In [117]:
# Print the imbalanced classification report
print("SMOTE Oversampler")
print("--------------------")
print("Imbalanced Classification Report:")
print(classification_report_imbalanced(y_test, y_pred_s))
print("Balanced Accuracy Score:")
balanced_accuracy_score(y_test, y_pred_s)

SMOTE Oversampler
--------------------
Imbalanced Classification Report:
                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.57      0.66      0.02      0.62      0.38        87
          1       1.00      0.66      0.57      0.79      0.62      0.38     17118

avg / total       0.99      0.66      0.58      0.79      0.62      0.38     17205

Balanced Accuracy Score:


0.6172137818227235

# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [105]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled_c, y_resampled_c = cc.fit_resample(X_train, y_train)
Counter(y_resampled_c)

Counter({0: 260, 1: 260})

In [106]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled_c, y_resampled_c)

LogisticRegression(random_state=1)

In [107]:
# Calculated the balanced accuracy score
y_pred_c = model.predict(X_test)

balanced_accuracy_score(y_test, y_pred_c)

0.5065045465350044

In [108]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_c)

array([[  51,   36],
       [9812, 7306]])

In [118]:
# Print the imbalanced classification report
print("Cluster Centroids Undersampler")
print("--------------------")
print("Imbalanced Classification Report:")
print(classification_report_imbalanced(y_test, y_pred_c))
print("Balanced Accuracy Score:")
balanced_accuracy_score(y_test, y_pred_c)

Cluster Centroids Undersampler
--------------------
Imbalanced Classification Report:
                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.59      0.43      0.01      0.50      0.25        87
          1       1.00      0.43      0.59      0.60      0.50      0.25     17118

avg / total       0.99      0.43      0.59      0.59      0.50      0.25     17205

Balanced Accuracy Score:


0.5065045465350044

# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [110]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=1)
X_resampled_smn, y_resampled_smn = smote_enn.fit_resample(X_train, y_train)
Counter(y_resampled_smn)

Counter({0: 51344, 1: 46561})

In [111]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled_smn, y_resampled_smn)

LogisticRegression(random_state=1)

In [112]:
# Calculated the balanced accuracy score
y_pred_smn = model.predict(X_test)

balanced_accuracy_score(y_test, y_pred_smn)

0.6471127387585562

In [113]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_smn)

array([[  64,   23],
       [7556, 9562]])

In [120]:
# Print the imbalanced classification report
print("Combination Sampling with SMOTEENN")
print("--------------------")
print("Imbalanced Classification Report:")
print(classification_report_imbalanced(y_test, y_pred_smn))
print("Balanced Accuracy Score:")
balanced_accuracy_score(y_test, y_pred_smn)

Combination Sampling with SMOTEENN
--------------------
Imbalanced Classification Report:
                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.74      0.56      0.02      0.64      0.42        87
          1       1.00      0.56      0.74      0.72      0.64      0.40     17118

avg / total       0.99      0.56      0.73      0.71      0.64      0.40     17205

Balanced Accuracy Score:


0.6471127387585562